In [ ]:
! pip install datasets

In [ ]:
! pip install transformers

In [9]:
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd
from torchtext.data.utils import get_tokenizer
from transformers import AutoTokenizer, BertTokenizer
import string
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Load Database

In [11]:
dataset = load_dataset("persiannlp/parsinlu_translation_fa_en")
dataset

Found cached dataset parsinlu_translation_fa_en (/Users/haniyeh/.cache/huggingface/datasets/persiannlp___parsinlu_translation_fa_en/parsinlu-repo/1.0.0/43e422c8c7004e6912c9da2ec24a7401fe1c292f477ab458b62624526a246b4a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 1622280
    })
    test: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 47744
    })
    validation: Dataset({
        features: ['source', 'targets', 'category'],
        num_rows: 2137
    })
})

In [12]:
df_train = pd.DataFrame(dataset['train'], columns = ['source','targets'])
df_test = pd.DataFrame(dataset['test'], columns = ['source','targets'])
df_val = pd.DataFrame(dataset['validation'], columns = ['source', 'targets'])

In [13]:
df_train['target'] = df_train['targets'].map(lambda x: x[0])
df_test['target'] = df_train['targets'].map(lambda x: x[0])
df_val['target'] = df_train['targets'].map(lambda x: x[0])

In [14]:
df_train.head()

,source,targets,target
0,بلاگر مصری عبدل منعم محمود (عربی) پس از آزاد ش...,[Due Thank You note by Egyptian blogger Abdel ...,Due Thank You note by Egyptian blogger Abdel M...
1,وی از تمامی بلاگرها، سازمان‌ها و افرادی که از ...,[He thanked all fellow bloggers and organizati...,He thanked all fellow bloggers and organizatio...
2,وی همچنین از دریافت تعداد بسیار زیادی پیام تبر...,[He was extremely surprised and happy to recei...,He was extremely surprised and happy to receiv...
3,وی گفت در ابتدا قصد داشت درباره تجربه‌اش در زن...,[He had the intention to write about his last ...,He had the intention to write about his last e...
4,وی در پایان نوشته‌اش می‌نویسد آزادی واقعا بها ...,[He concludes his post saying that freedom is ...,He concludes his post saying that freedom is t...


# Pre Process

In [15]:
def remove_punctuation(s):
    punctuationFree="".join([i for i in s if i not in string.punctuation])
    return punctuationFree

In [16]:
def preprocess_persian_string(s):
    punctuationFree = remove_punctuation(s)
    return punctuationFree

In [17]:
def preprocess_english_string(s):
    punctuationFree = remove_punctuation(s)
    return punctuationFree.lower().strip()

# Tokenizer

In [18]:
fa_model = "HooshvareLab/bert-fa-base-uncased"
fa_tokenizer = BertTokenizer.from_pretrained(fa_model)

In [19]:
en_model = "bert-base-uncased"
en_tokenizer = BertTokenizer.from_pretrained(en_model)

In [20]:
FA_SPACIAL_TOKENS = fa_tokenizer.all_special_tokens
FA_SPACIAL_TOKENS_IDS = fa_tokenizer.all_special_ids

print(FA_SPACIAL_TOKENS)
print(FA_SPACIAL_TOKENS_IDS)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 4, 0, 2, 3]


In [21]:
EN_SPACIAL_TOKENS = en_tokenizer.all_special_tokens
EN_SPACIAL_TOKENS_IDS = en_tokenizer.all_special_ids

print(EN_SPACIAL_TOKENS)
print(EN_SPACIAL_TOKENS_IDS)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[100, 102, 0, 101, 103]


# Dataloader

In [22]:
def generate_data(df):
    data = []
    for index, row in df.iterrows():
        source = preprocess_persian_string(row['source'])
        target = preprocess_english_string(row['target'])

        fa_tensor_ = torch.tensor(fa_tokenizer(source).input_ids)
        en_tensor_ = torch.tensor(en_tokenizer(target).input_ids)
        data.append((fa_tensor_, en_tensor_))
    return data

In [33]:
train_data = generate_data(df_train)
test_data = generate_data(df_test)
val_data = generate_data(df_val)

In [29]:
train = pd.DataFrame(train_data)
train.to_csv('train.csv', index=False, header=False)

val = pd.DataFrame(val_data)
val.to_csv('val.csv', index=False, header=False)

test = pd.DataFrame(test_data)
test.to_csv('val.csv', index=False, header=False)

In [34]:
train_data[0]

(tensor([    2, 80928, 11391, 45770,  6625,  2015,  5931,  5279,  2965,  2791,
          3912, 20633,  2791,  5511, 13477,  3455,  5071,  2847,  2803,  1064,
          2842,  3912,  6794,  1078,  5279,  3229,  2830,     4]),
 tensor([  101,  2349,  4067,  2017,  3602,  2011,  6811, 28205, 19935,  2884,
         12256,  6633, 27278,  2206,  2010,  2713,  2013,  3827,  2002,  2626,
          2010,  2034,  9927,  4159,  1523,  9617, 20912, 18663,  2078,  1529,
          1045,  2572,  2489,  1524,   102]))

In [35]:
FA_TRAIN_VOCAB_SIZE = max([max(data[0]) for data in train_data])
FA_VALIDATION_VOCAB_SIZE = max([max(data[0]) for data in val_data])
FA_TEST_VOCAB_SIZE = max([max(data[0]) for data in test_data])

In [36]:
EN_TRAIN_VOCAB_SIZE = max([max(data[1]) for data in train_data])
EN_VALIDATION_VOCAB_SIZE = max([max(data[1]) for data in val_data])
EN_TEST_VOCAB_SIZE = max([max(data[1]) for data in test_data])

In [37]:
FA_TRAIN_MAX_LEN = max([len(data[0]) for data in train_data])
FA_VALIDATION_MAX_LEN = max([len(data[0]) for data in val_data])
FA_TEST_MAX_LEN = max([len(data[0]) for data in test_data])

In [38]:
EN_TRAIN_MAX_LEN = max([len(data[1]) for data in train_data])
EN_VALIDATION_MAX_LEN = max([len(data[1]) for data in val_data])
EN_TEST_MAX_LEN = max([len(data[1]) for data in test_data])

In [39]:
SOURCE_MAX_LEN = max([FA_TRAIN_MAX_LEN,FA_VALIDATION_MAX_LEN,FA_TEST_MAX_LEN])
TARGET_MAX_LEN = max([EN_TRAIN_MAX_LEN,EN_VALIDATION_MAX_LEN,EN_TEST_MAX_LEN])

In [40]:
TARGET_MAX_LEN

351

In [41]:
train_data[0]

(tensor([    2, 80928, 11391, 45770,  6625,  2015,  5931,  5279,  2965,  2791,
          3912, 20633,  2791,  5511, 13477,  3455,  5071,  2847,  2803,  1064,
          2842,  3912,  6794,  1078,  5279,  3229,  2830,     4]),
 tensor([  101,  2349,  4067,  2017,  3602,  2011,  6811, 28205, 19935,  2884,
         12256,  6633, 27278,  2206,  2010,  2713,  2013,  3827,  2002,  2626,
          2010,  2034,  9927,  4159,  1523,  9617, 20912, 18663,  2078,  1529,
          1045,  2572,  2489,  1524,   102]))

In [42]:
def generate_batch(data_batch):

    data = []
    for idx, (fa, en) in enumerate(data_batch):
        if len(fa) < TARGET_MAX_LEN:
            padding_tensor =  torch.zeros(TARGET_MAX_LEN - len(fa),dtype=int)
            padding_tensor[:] = FA_SPACIAL_TOKENS_IDS[2]
            fa = torch.cat((fa,padding_tensor), dim=0) 
        
        if len(en) < TARGET_MAX_LEN:
            padding_tensor =  torch.zeros(TARGET_MAX_LEN - len(en),dtype=int)
            padding_tensor[:] = EN_SPACIAL_TOKENS_IDS[2]
            en = torch.cat((en,padding_tensor), dim=0) 
        data.append( (fa,en) )
            
    return data

In [43]:
train_padded = generate_batch(train_data)

In [44]:
val_padded = generate_batch(val_data)

In [45]:
test_padded = generate_batch(test_data)

In [46]:
BATCH_SIZE = 16

train_iter = DataLoader(train_padded, batch_size=BATCH_SIZE, shuffle=True)
valid_iter = DataLoader(val_padded, batch_size=BATCH_SIZE, shuffle=False)
test_iter = DataLoader(test_padded, batch_size=BATCH_SIZE, shuffle=False)

# Model

In [47]:
SOURCE_VOCAB_SIZE = int(max([FA_TEST_VOCAB_SIZE, FA_TRAIN_VOCAB_SIZE, FA_VALIDATION_VOCAB_SIZE]))
TARGET_VOCAB_SIZE = int(max([EN_TEST_VOCAB_SIZE, EN_TRAIN_VOCAB_SIZE, EN_VALIDATION_VOCAB_SIZE]))
NUM_LAYER = 6
# BATCH_SIZE = 32
EMBED_DIMENSION = 512
NUMBER_HEADS = 8

In [79]:
%run transformer.ipynb

torch.Size([32, 12]) torch.Size([32, 12])
torch.Size([32, 12])
here
here
here
here
here
here


/var/folders/pb/4t256kf1739f02pn7ldh71lw0000gn/T/ipykernel_74043/531710156.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(self.fc_layer(x))


In [80]:
class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, target_vocab_size, embed_dim, seq_len, num_layers, expansion_factor=4, n_heads=8):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.transformer = Transformer(embed_dim,src_vocab_size,target_vocab_size,seq_len,num_layers, expansion_factor, n_heads)
        self.output = nn.Linear(embed_dim, target_vocab_size)

    def forward(self,src, tgt):
        print("tgt",tgt.shape)
        output = self.transformer(src, tgt)
        
        return self.output(output)

In [81]:
def train_transformer(model, iterator, optimizer, loss_fn, device, clip=None):
    model.train()
        
    epoch_loss = 0
    with tqdm(total=len(iterator), leave=False) as t:
        for i, (src, tgt) in enumerate(iterator):
            src = src.to(device)
            tgt = tgt.to(device)
           
            
            # Create tgt_inp and tgt_out (which is tgt_inp but shifted by 1)
            tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]
            print("tgt_inp", tgt_inp.shape)
            optimizer.zero_grad()
                
            output = model(src=src, tgt=tgt_inp)
            
            loss = loss_fn(output.view(-1, output.shape[2]),
                           tgt_out.view(-1))
            
            loss.backward()
            
            # if clip is not None:
            #     nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step()
            epoch_loss += loss.item()
            
            avg_loss = epoch_loss / (i+1)
            t.set_postfix(loss='{:05.3f}'.format(avg_loss),
                          ppl='{:05.3f}'.format(np.exp(avg_loss)))
            t.update()
            
    return epoch_loss / len(iterator)

In [82]:
def evaluate_transformer(model, iterator, loss_fn, device):
    model.eval()
        
    epoch_loss = 0
    with torch.no_grad():
        with tqdm(total=len(iterator), leave=False) as t:
            for i, (src, tgt) in enumerate(iterator):
                src = src.to(device)
                tgt = tgt.to(device)
                
                # Create tgt_inp and tgt_out (which is tgt_inp but shifted by 1)
                tgt_inp, tgt_out = tgt[:, :-1], tgt[:, 1:]

                output = model(src=src, tgt=tgt_inp)
                
                loss = loss_fn(output.view(-1, output.shape[2]),
                               tgt_out.view(-1))
                
                epoch_loss += loss.item()
                
                avg_loss = epoch_loss / (i+1)
                t.set_postfix(loss='{:05.3f}'.format(avg_loss),
                              ppl='{:05.3f}'.format(np.exp(avg_loss)))
                t.update()
    
    return epoch_loss / len(iterator)

In [83]:
def count_params(model, return_int=False):
    params = sum([torch.prod(torch.tensor(x.shape)).item() for x in model.parameters() if x.requires_grad])
    if return_int:
        return params
    else:
        print("There are {:,} trainable parameters in this model.".format(params))

In [84]:
transformer = TransformerModel(src_vocab_size=SOURCE_VOCAB_SIZE, target_vocab_size=TARGET_VOCAB_SIZE, embed_dim=256, seq_len=TARGET_MAX_LEN, num_layers=2, expansion_factor=4, n_heads=8)

transformer = transformer.to(device)

In [85]:
count_params(transformer)

There are 52,588,914 trainable parameters in this model.


In [86]:
xf_optim = torch.optim.AdamW(transformer.parameters(), lr=1e-4)

In [87]:
PAD_IDX = FA_SPACIAL_TOKENS_IDS[2]
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [88]:
%%time
N_EPOCHS = 50
CLIP = 16 # clipping value, or None to prevent gradient clipping
EARLY_STOPPING_EPOCHS = 5
SAVE_DIR = ''
import os
    
model_path = os.path.join(SAVE_DIR, 'transformer_en_fr.pt')
transformer_metrics = {}
best_valid_loss = float("inf")
early_stopping_count = 0
for epoch in tqdm(range(N_EPOCHS), desc="Epoch"):
    train_loss = train_transformer(transformer, train_iter, xf_optim, loss_fn, device, clip=CLIP)
    valid_loss = evaluate_transformer(transformer, valid_iter, loss_fn, device)
    
    if valid_loss < best_valid_loss:
        tqdm.write(f"Checkpointing at epoch {epoch + 1}")
        best_valid_loss = valid_loss
        torch.save(transformer.state_dict(), model_path)
        early_stopping_count = 0
    elif epoch > EARLY_STOPPING_EPOCHS:
        early_stopping_count += 1
    
    transformer_metrics[epoch+1] = dict(
        train_loss = train_loss,
        train_ppl = np.exp(train_loss),
        valid_loss = valid_loss,
        valid_ppl = np.exp(valid_loss)
    )
    
    if early_stopping_count == EARLY_STOPPING_EPOCHS:
        tqdm.write(f"Early stopping triggered in epoch {epoch + 1}")
        break

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/101393 [00:00<?, ?it/s]

tgt_inp torch.Size([16, 350])
tgt torch.Size([16, 350])
torch.Size([16, 351])


RuntimeError: The size of tensor a (350) must match the size of tensor b (351) at non-singleton dimension 3